In [1]:
import pandas as pd
import re
from nltk import ngrams
from form_keywords import form_keywords
gram_range = tuple((2, 4))
unlabelled_file = './data/dec.csv'
keyword_file = './data/jantodec.csv'

In [2]:
def remove(string):
    string = re.sub(r'([0-9])', r'', str(string))
    string = re.sub(r'[a-zA-z+]', r'', str(string))
    string = re.sub(r'[^\w\s]', r'', str(string))
    return string

In [3]:
def n_grams(string):
    all_grams = []
    string = re.sub(r'(.)', r'\g<1> ', str(string))
    string = string.split()
    for gram in range(gram_range[0], gram_range[1] + 1):
        for g in ngrams(string, n = gram):
            all_grams.append(''.join(list(g)))
    return all_grams

In [4]:
df = pd.read_csv(unlabelled_file)
unlabelled = df.drop(df[df['category_id'] != '未分類'].index)

In [5]:
df.head()

,shop_id,order_id,variant_title,sku,total_price,product_id,plan,name,price,title,category_id
0,16132,7442041,成人,MAA0005-4,455,32319783,enterprise,styleilife,109.0,【STYLEi 醫療口罩】聖誕禮物盒系列｜湖水綠-成人MIT&MD雙鋼印/一盒8入,503724
1,16132,7439504,成人,MAA0005-4,1059,32319783,enterprise,styleilife,109.0,【STYLEi 醫療口罩】聖誕禮物盒系列｜湖水綠-成人MIT&MD雙鋼印/一盒8入,503724
2,16132,7439914,大人,MAA0004-4,274,32319639,enterprise,styleilife,109.0,【STYLEi 醫療口罩】聖誕動物系列｜企鵝忙派對-成人MIT&MD雙鋼印/一盒8入,503724
3,16132,7443818,大人,MAA0004-4,274,32319639,enterprise,styleilife,109.0,【STYLEi 醫療口罩】聖誕動物系列｜企鵝忙派對-成人MIT&MD雙鋼印/一盒8入,503724
4,16132,7439504,大人,MAA0004-4,1059,32319639,enterprise,styleilife,109.0,【STYLEi 醫療口罩】聖誕動物系列｜企鵝忙派對-成人MIT&MD雙鋼印/一盒8入,503724


In [6]:
fk = form_keywords(keyword_file, keyword_num = 30, n = gram_range)
keyword_dict = fk.fit()
#Move 'Arts & Entertainment' to last of dictionary due to the fact that it has more noise.
keyword_dict.pop('Media', None)
keyword_dict.pop('Mature', None)
a_list = keyword_dict['Arts & Entertainment']
keyword_dict.pop('Arts & Entertainment', None)
keyword_dict['Arts & Entertainment'] = a_list

/Users/jackson/.pyenv/versions/3.8.3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
category = []
for row in unlabelled.values:
    counting = []
    string = str(row[7]) + str(row[9])
    string = remove(string)
    patterns = n_grams(string)
    for cat in keyword_dict.keys():
        cat_count = 0
        for pat in patterns:
#             cat_count += len(re.findall(pat, ' '.join(keyword_dict[cat])))
            if pat in keyword_dict[cat]:
                cat_count += 1
        counting.append(tuple((cat, cat_count)))
    counting = sorted(counting, key = lambda x : x[1])
    if counting[-1][1] > 0:
        category.append(counting[-1][0])
    else:
        category.append('未分類')

In [8]:
c = 0
for cat in category:
    if cat == '未分類':
        c += 1

In [9]:
c

120724

In [10]:
len(category)

228725

In [11]:
c / len(category)

0.5278128757241228

In [12]:
unlabelled['cname'] = category
lab = unlabelled.drop(unlabelled[unlabelled['cname'] == '未分類' ].index)

In [13]:
# import pickle
# with open('./newly_labelled.pickle', 'wb') as file:
#     pickle.dump(lab, file)

In [14]:
lab.to_csv('./newly_labelled.csv', encoding = 'utf_8_sig')

In [15]:
lab.head()

,shop_id,order_id,variant_title,sku,total_price,product_id,plan,name,price,title,category_id,cname
23,16167,7443563,2盒組 免運,20200723-1,719,31812381,enterprise,BIC® 刮鬍刀,739.0,BIC® 極淨5,未分類,Arts & Entertainment
24,16167,7441184,3盒 優惠組 (免運),20200423-2,1080,31812381,enterprise,BIC® 刮鬍刀,1080.0,BIC® 極淨5,未分類,Arts & Entertainment
25,16167,7445591,1盒 (1刀柄4刀片/盒),20200423-1,738,31812381,enterprise,BIC® 刮鬍刀,379.0,BIC® 極淨5,未分類,Arts & Entertainment
26,16167,7439959,1盒 (1刀柄4刀片/盒),20200423-1,459,31812381,enterprise,BIC® 刮鬍刀,379.0,BIC® 極淨5,未分類,Arts & Entertainment
27,16167,7439391,1盒 (1刀柄4刀片/盒),20200423-1,578,31812381,enterprise,BIC® 刮鬍刀,379.0,BIC® 極淨5,未分類,Arts & Entertainment


In [12]:
#dec: 5 -> 0.85, 10 -> 0.76, 30 -> 0.63

In [13]:
#seasons 10 -> 0.79, 30 -> 0.61

In [14]:
#dec + name : 10 -> 0.71, 

In [15]:
#dec + name (seasons for keyword generation): 10 -> 0.69

In [16]:
#dec + name (julytodec): 30 -> 0.53

In [18]:
unlabelled[100:150]

,shop_id,order_id,variant_title,sku,total_price,product_id,plan,name,price,title,category_id,cname
139,16076,7446416,1瓶組,4713233982253,3148,31790431,enterprise,cleanclean,120.0,(加購) 清潔皂50ml,未分類,Hardware
140,16076,7440652,1瓶組,4713233982253,919,31790431,enterprise,cleanclean,120.0,(加購) 清潔皂50ml,未分類,Hardware
141,16076,7447717,1瓶組,4713233980907,2209,31950450,enterprise,cleanclean,330.0,(加購) 淨手護100ml,未分類,未分類
142,16076,7447399,1瓶組,4713233982574,890,31790430,enterprise,cleanclean,150.0,(加購) 淨潤膚 50ml,未分類,未分類
143,16076,7448109,1瓶組,4713233982574,949,31790430,enterprise,cleanclean,150.0,(加購) 淨潤膚 50ml,未分類,未分類
144,16076,7447399,1瓶 (250ml),4713233981096,890,31769644,enterprise,cleanclean,330.0,淨潤膚/輕巧瓶,未分類,未分類
145,16076,7447325,1瓶 (250ml),4713233981096,1859,31769644,enterprise,cleanclean,330.0,淨潤膚/輕巧瓶,未分類,未分類
146,16076,7447573,1瓶 (250ml),4713233981096,1728,31769644,enterprise,cleanclean,330.0,淨潤膚/輕巧瓶,未分類,未分類
147,16076,7448102,1瓶 (250ml),4713233981096,1960,31769644,enterprise,cleanclean,330.0,淨潤膚/輕巧瓶,未分類,未分類
148,16076,7447680,3瓶 優惠組,20201006-3,2020,31769056,enterprise,cleanclean,380.0,淨淨剋菌液/隨身瓶,未分類,未分類


In [25]:
#calculate unlabelled keywords and retrieve corresponding shop id
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
import pickle
unlabelled = unlabelled.drop(unlabelled[unlabelled['cname'] != '未分類' ].index)
un_docs = []
for row in unlabelled.values:
    string = row[7] + row[9]
    string = remove(string)
    string = re.sub(r'(.)', r'\g<1> ', string)
    un_docs.append(string)
cv = CountVectorizer(max_features = 30, 
    tokenizer = lambda x: x.split(), ngram_range = (gram_range[0], gram_range[1]))
word_count_vector = cv.fit_transform(un_docs)
unlabelled_keywords = [word.replace(' ', '') for word in cv.vocabulary_.keys()]
shop_id = []
for row in unlabelled.values:
    string = str(row[7]) + str(row[9])
    string = remove(string)
    patterns = n_grams(string)
    for pat in patterns:
        if pat in unlabelled_keywords:
            shop_id.append(tuple((row[0], pat)))  #append shop id
idx = [s_id for (s_id, pat) in shop_id ]


shopid_to_matchcount = defaultdict(int)
for x in idx:
    shopid_to_matchcount[int(x)] += 1

a = sorted(shopid_to_matchcount, key = lambda x: shopid_to_matchcount[x])
# with open('./shopid_to_matchcount.pickle', 'wb') as out:
#     pickle.dump(shopid_to_matchcount, out)
# with open('./shopid_unlabelledmatch.pickle', 'wb') as out:
#     pickle.dump(a, out)
shop_to_pat = defaultdict(list)
for (shop, pat) in shop_id:
    shop_to_pat[shop].append(pat)
df = []
for key in shop_to_pat.keys():
    row = []
    row.append(key)
    row.append(shop_to_pat[key])
    df.append(row)
df = pd.DataFrame(df, columns = ['shop', 'patterns'])
df.to_csv('./shop_to_pat.csv')

In [26]:
unlabelled_keywords

['限定',
 '官方',
 '方購',
 '購物',
 '物網',
 '官方購',
 '方購物',
 '購物網',
 '官方購物',
 '方購物網',
 '網站',
 '健康',
 '商城',
 '官網',
 '保溫',
 '溫袋',
 '袋綠',
 '保溫袋',
 '溫袋綠',
 '保溫袋綠',
 '姆士',
 '春子',
 '李大',
 '大娘',
 '買菜',
 '菜網',
 '李大娘',
 '娘買菜',
 '買菜網',
 '娘買菜網']

In [28]:
cv.vocabulary_

{'限 定': 29,
 '官 方': 9,
 '方 購': 13,
 '購 物': 27,
 '物 網': 21,
 '官 方 購': 10,
 '方 購 物': 14,
 '購 物 網': 28,
 '官 方 購 物': 11,
 '方 購 物 網': 15,
 '網 站': 22,
 '健 康': 3,
 '商 城': 4,
 '官 網': 12,
 '保 溫': 0,
 '溫 袋': 19,
 '袋 綠': 24,
 '保 溫 袋': 1,
 '溫 袋 綠': 20,
 '保 溫 袋 綠': 2,
 '姆 士': 6,
 '春 子': 16,
 '李 大': 17,
 '大 娘': 5,
 '買 菜': 25,
 '菜 網': 23,
 '李 大 娘': 18,
 '娘 買 菜': 7,
 '買 菜 網': 26,
 '娘 買 菜 網': 8}